In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import preprocessing

In [2]:
# Import dataset with panda
df = pd.read_csv('output.csv')
features_considered = ['embb_req', 'embb_giv', 'embb_true', 'urllc_req', 'urllc_giv', 'urllc_true','miot_req', 'miot_giv', 'miot_true','mmtc_req', 'mmtc_giv', 'mmtc_true','voice_req', 'voice_giv', 'voice_true', 'band_rem']
features = df[features_considered]
features.index = df['round']
features.head()


,embb_req,embb_giv,embb_true,urllc_req,urllc_giv,urllc_true,miot_req,miot_giv,miot_true,mmtc_req,mmtc_giv,mmtc_true,voice_req,voice_giv,voice_true,band_rem
round,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20000000000
1,74658574351,11370057492,17867115683,462471153,461673311,462105308,347772505,347772505,347772505,439177523,439177523,439177523,608224277,102000000,373615493,7279319169
2,72936751644,11497085964,18745456595,111169995,111169995,111169995,124473091,124473091,124473091,126376030,126376030,126376030,579979842,115000000,427544446,8025894920
3,69020442187,11477653657,18451665215,279575826,279575826,279575826,236468628,236468628,236468628,226172103,226172103,226172103,499106356,121000000,428011871,7659129786
4,62180480903,11312809470,18633767756,261703197,260293311,261157940,191265801,191265801,191265801,250010530,250010530,250010530,380045936,121000000,378043030,7864620888


In [3]:
# Parametes 
TRAIN_SPLIT = 240000
tf.random.set_seed(13)
past_history = 1
future_target = 0
step = 1
BATCH_SIZE = 32 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 300000 # >= len dataset for perfetct shuffling
EVALUATION_INTERVAL = 7500 # steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
VALIDATION_STEPS = 1875 # validation_steps = TotalvalidationSamples / ValidationBatchSize
EPOCHS = 20
bs_capacity = 20000000000


In [4]:
# Normalize
dataset = features.values
dataset = dataset / bs_capacity

In [5]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)


In [6]:
# Create validation and training set
x_train, y_train = multivariate_data(dataset[1:, (0, 1, 3, 4, 6, 7, 9, 10, 12 ,13, 15)], dataset[1:, (2, 5, 8, 11, 14)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 1, 3, 4, 6, 7, 9, 10, 12 ,13, 15)], dataset[1:, (2, 5, 8, 11, 14)], TRAIN_SPLIT, None, past_history, future_target, step)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [7]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

# Compile RNN
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

# Train RNN
history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 7500 steps, validate for 1875 steps
Epoch 1/20
7500/7500 [==============================] - 71s 10ms/step - loss: 0.0121 - val_loss: 0.0156
Epoch 2/20
7500/7500 [==============================] - 61s 8ms/step - loss: 0.0098 - val_loss: 0.0148
Epoch 3/20
7500/7500 [==============================] - 62s 8ms/step - loss: 0.0092 - val_loss: 0.0139
Epoch 4/20
7500/7500 [==============================] - 76s 10ms/step - loss: 0.0089 - val_loss: 0.0136
Epoch 5/20
7500/7500 [==============================] - 59s 8ms/step - loss: 0.0086 - val_loss: 0.0130
Epoch 6/20
7500/7500 [==============================] - 62s 8ms/step - loss: 0.0084 - val_loss: 0.0132
Epoch 7/20
7500/7500 [==============================] - 64s 9ms/step - loss: 0.0082 - val_loss: 0.0126
Epoch 8/20
7500/7500 [==============================] - 60s 8ms/step - loss: 0.0081 - val_loss: 0.0126
Epoch 9/20
7500/7500 [==============================] - 66s 9ms/step - loss: 0.0081 - val_loss: 0.0121
Epoch 10/20
7500/7500 [==

In [8]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[35881556553, 8318124338, 310731060, 310731060, 268358393, 268358393, 197834085, 197834085, 385305779, 64000000, 10840952124]]]
test = np.array(test)/bs_capacity

prediction = model.predict(test)
print(prediction[0])
print(y_train[0]) # because I took the input of the first values in x_train for test


[0.9521095  0.00750191 0.00699863 0.00722724 0.00777826]
[0.94652989 0.00816619 0.00827225 0.01066902 0.00550193]
[0.9461166  0.00769644 0.00719851 0.00749568 0.010278  ]
[0.93727283 0.0055585  0.00622365 0.0063188  0.02137722]


In [9]:
# Save trained model
model.save("model.h5")


In [ ]:
[0.9464359  0.0085995  0.00715667 0.00964167 0.01064707]
[0.93727283 0.0055585  0.00622365 0.0063188  0.02137722]